<a href="https://colab.research.google.com/github/srihasdama/4095_Final_Project/blob/main/Copy_of_RedditProject1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import numpy as np
import pandas as pd

In [101]:
!pip install datasets
# FYI the accellerate -u is very important to avoid version issues with torch
!pip install transformers
!pip install accelerate -U

In [102]:
from datasets import load_dataset

In [103]:
dataset = load_dataset("fddemarco/pushshift-reddit-comments", streaming = True)

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

In [104]:
print(dataset)

IterableDatasetDict({
    train: IterableDataset({
        features: ['author', 'body', 'controversiality', 'created_utc', 'link_id', 'score', 'subreddit', 'subreddit_id', 'id'],
        n_shards: 50
    })
})


In [105]:
it = iter(dataset['train'])


In [106]:
extract_data = next(it)
#print(extract_data)

print(extract_data['body'])
#testing if we can get a sample comment

isn ' t this against the first amendment ? doesn ' t the first amendment give us the right to assemble and protest ?


In [107]:
data = {
    'column1_name': ['text'],
    'column2_name': ['score'],
    'column3_name': ['labels']
}

df = pd.DataFrame(data)

In [109]:
# Initialize an empty DataFrame with columns
df = pd.DataFrame(columns=['text', 'score', 'labels'])

nsfw_keywords = ['nsfw', 'adult', 'sex', 'porn', 'xxx', 'gonewild', 'nudes', 'slut', 'WTF',
'hentai', 'cum', 'rule34', 'pussy', 'onlyfans', 'milf', 'nude', 'boob']

mask = -df['labels'].str.contains('|'.join(nsfw_keywords))
df = df[mask]

for i in range(100000):  # only 10,000 rows
    extract_data = next(it)
    text = extract_data['body']
    score = extract_data['score']
    labels = extract_data['subreddit']

    new_row_data = {'text': text, "score": score, "labels": labels}
    df.loc[len(df)] = new_row_data






In [110]:
df.head()

,text,score,labels
0,dosent go to party still has to find pants,2,funny
1,"hear about that fire at the circus ? yeah , wa...",3,RandomActsOfGaming
2,i was watching the london marathon and saw one...,2,RandomActsOfGaming
3,"if day9 does stream , i ' ll be white - knight...",3,day9
4,i ' m curious how this plays out once all the ...,2,politics


In [111]:
id2label = dict(enumerate(set(df['labels'])))

In [ ]:
id2label

In [113]:
df.head()


,text,score,labels
0,dosent go to party still has to find pants,2,funny
1,"hear about that fire at the circus ? yeah , wa...",3,RandomActsOfGaming
2,i was watching the london marathon and saw one...,2,RandomActsOfGaming
3,"if day9 does stream , i ' ll be white - knight...",3,day9
4,i ' m curious how this plays out once all the ...,2,politics


In [114]:
from transformers import AutoTokenizer, RobertaForSequenceClassification
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Load roberta model and tokenizer filling out the code below based on the documentation HINT: Click on on use in transformers and the code will be there
#model_name = "Roberta model"# code unnessisary but I like to use it to store the string name to make the code cleaner
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = len(id2label))


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [115]:
nan_rows = df[df.isna().T.any().T]
nan_rows.head()

,text,score,labels


In [116]:
null_rows = df[df.isnull().any(axis=1)]
null_rows.head()

,text,score,labels


In [117]:
if np.isinf(df['score']).any():
  print("HELLO")

In [118]:
print(df.dtypes)

text      object
score      int64
labels    object
dtype: object


In [119]:
df.head()

,text,score,labels
0,dosent go to party still has to find pants,2,funny
1,"hear about that fire at the circus ? yeah , wa...",3,RandomActsOfGaming
2,i was watching the london marathon and saw one...,2,RandomActsOfGaming
3,"if day9 does stream , i ' ll be white - knight...",3,day9
4,i ' m curious how this plays out once all the ...,2,politics


In [133]:
label_counts = df['labels'].value_counts()
labels_to_remove = label_counts[label_counts < 2].index
df = df[~df['labels'].isin(labels_to_remove)]

In [134]:
df['labels'].value_counts()

labels
AskReddit            12481
pics                  6549
gaming                5157
atheism               4432
IAmA                  3848
                     ...  
notporn                  2
Slender_Man              2
TeaParty                 2
GraffitiContests         2
NewcastleUponTyne        2
Name: count, Length: 1512, dtype: int64

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99332 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    99332 non-null  object
 1   score   99332 non-null  int64 
 2   labels  99332 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.0+ MB


In [137]:
from sklearn.model_selection import train_test_split

#TODO: Complete this line using the train_test_split function and set the test size to .21
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['labels'], test_size=0.21, stratify = df['labels'])


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
# Reset the index
train_texts, train_labels = train_texts.reset_index(drop=True), train_labels.reset_index(drop=True)
val_texts, val_labels = val_texts.reset_index(drop=True), val_labels.reset_index(drop=True)

In [138]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, return_tensors = "pt")
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, return_tensors = "pt")

In [139]:
id2label_reversed = {v: k for k, v in id2label.items()}

In [140]:
id2label_reversed #word:number

{'lxnews': 0,
 'EarthPorn': 1,
 'rum': 2,
 'aves': 3,
 'sheffield': 4,
 'Metal': 5,
 'metalmusicians': 6,
 'Amazing2': 7,
 'Boxer': 8,
 'discworld': 9,
 'RockClimbing': 10,
 'otaku': 11,
 'gaymersgonewild': 12,
 'machinima': 13,
 'Hunting': 14,
 'bleach': 15,
 'Torontobluejays': 16,
 'secretsanta': 17,
 'television': 18,
 'mylittlehuman': 19,
 'ElleFanning': 20,
 'teamlast': 21,
 'GoliathWorld': 22,
 'rape': 23,
 'barefoot': 24,
 'listentous': 25,
 'foundfootage': 26,
 'tennis': 27,
 'TreesFortress2': 28,
 'jobs': 29,
 'Entrepreneur': 30,
 'xbox360': 31,
 'PirateParty': 32,
 'roomdetective': 33,
 'nintendo': 34,
 'xbox': 35,
 'Thailand': 36,
 'arresteddevelopment': 37,
 'AppalachianTrail': 38,
 'Reprap': 39,
 'crabdancemc': 40,
 'photography': 41,
 '4chan': 42,
 'winterporn': 43,
 'Ultralight': 44,
 'caps': 45,
 'whatisthisthing': 46,
 'gay': 47,
 'cigars': 48,
 'Favors': 49,
 'apathy': 50,
 'BostonBruins': 51,
 'rpjh': 52,
 'classicalmusic': 53,
 'videogamebabes': 54,
 'EnterShikari':

In [141]:
import torch as pt

class SpamDataset(pt.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.values

    def __getitem__(self, idx):
        item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}
        if len(self.labels) > 0:  # Check if labels array has any elements
            labels = self.labels[idx]
            #print(labels)
            labels_converted = id2label_reversed[labels]
            #print(labels_converted)
            item["labels"] = pt.tensor(labels_converted)
        return item
    def __len__(self):
        print(len(self.labels))
        return len(self.labels)

def list_of_dicts_to_dict_of_lists(d):
  dic = d[0]
  keys = dic.keys()
  values = [dic.values() for dic in d]
  return {k: list(v) for k,v in zip(keys, zip(*values))}

In [142]:
Train_Dataset = SpamDataset(train_encodings, train_labels)


In [143]:
Val_Dataset = SpamDataset(val_encodings, val_labels)

In [144]:
!pip install datasets transformers torch evaluate nltk rouge_score

In [145]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate


In [146]:
from sklearn.metrics import accuracy_score

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


env: CUDA_LAUNCH_BLOCKING=1


'1'

In [149]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs, let's try 1 for now
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)



trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = Train_Dataset,
    eval_dataset = Val_Dataset,
    compute_metrics = compute_metrics
    #TODO: figure out what 5 params we need here. There should be 5 and the last is how we compute the metrics later . . .
)

78472


In [150]:
#set(id2label_reversed.values)

In [ ]:
# TODO: Train the model (HINT: this is 1 short line of code just calling the trainer)
trainer.train()

78472
78472
78472
78472
78472
78472
78472
78472
78472
78472
78472
78472
78472


<ipython-input-141-6fd8502b180c>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
10,7.706000
20,7.712700
30,7.690900
40,7.694500
50,7.686100
60,7.661700
70,7.698800
80,7.611300
90,7.498100
100,7.345300


<ipython-input-141-6fd8502b180c>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
#TODO use the evaluate method on the trainer to get and print the results. Feel free to look at Huggingface Docs
results = trainer.evaluate()

In [ ]:
# TODO: Save the model
trainer.save_model('my_model')

In [ ]:
# TODO: Now to finish off I want you to load the model you trained and saved and write a fake spam email for it mthen have the model classify it
model = AutoModelForSequenceClassification.from_pretrained("my_model")
msg = "I want to download minecraft mods"
msg_encodings = tokenizer(msg, truncation=True, padding=True, return_tensors = "pt")

outputs = model(**msg_encodings)
print(f"outputs: {outputs}")




In [ ]:
outputs.logits.shape

In [ ]:
len(id2label)

In [ ]:
probabilities = torch.softmax(outputs.logits[0], dim = 0 )
print(probabilities)

In [ ]:
sort_prob = probabilities.sort(descending = True)

In [ ]:
print(sort_prob)

In [ ]:
def print_out_top_x_probs(x,phrase):
  msg = phrase
  msg = "I want to download minecraft mods"
  print('Here are the top 10 most subreddits with similar')
  msg_encodings = tokenizer(msg, truncation=True, padding=True, return_tensors = "pt")
  probabilities = torch.softmax(outputs.logits[0], dim = 0 )
  sort_prob = probabilities.sort(descending = True)

  outputs = model(**msg_encodings)
  for n in range(x):
    sort_probs = probs
    tensor_probs = sort_probs.values[n]
    index = sort_probs.indices[n]
    sort_prob_index_int = index.item()
    subreddit_string = id2label[sort_prob_index_int]
    print(f'{n+1}. {subreddit_string} ({tensor_probs * 100}%)')

In [ ]:
sort_prob.indices[0].item()

In [ ]:
phrase = "what bird is this?"
print_out_top_x_probs(phrase)

In [ ]:
# it's time to graph!

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    return {
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy
    }


In [ ]:
import matplotlib.pyplot as plt

# this is fake data, replace with real data
epochs = 1
loss = [0.30, 0.24, 0.20]
f1_scores = [0.82, 0.85, 0.87]

plt.figure(figsize=(12, 5))

# Plotting loss
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, label='Training Loss', marker='o')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

# Plotting F1 Score
plt.subplot(1, 2, 2)
plt.plot(epochs, f1_scores, label='F1 Score', marker='o', color='r')
plt.title('F1 Score per Epoch')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()
